# Machine Learning Pitfalls Demonstrated via Enhancer Prediction

This notebook demonstrates several pitfalls of machine learning in genomics using the scikit-learn machine learning package.  Broad discussion of these pitfalls can be found in our review paper, "Navigating the Pitfalls of Applying Machine Learning in Genomics".  Some are well known and not specific to genomics or biology, while others have particular relevance in the field and may be under-appreciated.

To demonstrate various pitfalls, we build a binary classifier for enhancers in the K562 cell line.  Our formulation prioritizes demonstrating pitfalls over predictive performance.  Rows of our feature matrix are genomic regions with bi-directional transcription in one or more cell lines measured via Cap Analysis of Gene Expression (CAGE) performed by the [FANTOM5 consortium](https://fantom.gsc.riken.jp/5/datafiles/reprocessed/hg38_latest/extra/enhancer/).  Columns are binary features encoding if the region overlaps a statistically significant ChIP-seq peak for a histone modification or transcription factor (TF) assayed by ENCODE in K562.  A region is labeled 1 if its CAGE expression was non-zero in K562, and 0 if there was no expression in K562.

We first load features and labels, define several scikit-learn objects needed to demonstrate various pitfalls, and finally use those objects to fit models and evaluate performance.  Multiple performance metrics will be stored after each evaluation and summarized at the end for comparison.

#### - Sean Whalen, Gladstone Institutes, Pollard Lab

In [ ]:
import pandas as pd

from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.dummy import *
from sklearn.feature_selection import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from statsmodels.stats.proportion import proportions_ztest

For simplicity, the feature matrix was pre-generated and is loaded from a Feather file, which is a fast binary format for column-based data.  The hg38 genomic coordinates for each region are stored in a column named `coord`, which is later set as the index of the data frame so that the columns only contain features.

To better demonstrate some pitfalls, we filter the data to include regions from chromosomes 1 and 2 only.

In [ ]:
features = pd.read_feather('../data/cage-k562-features-hg38.feather')
features = features[features['coord'].str.contains(r'chr[12]:')]
features.set_index('coord', inplace = True)
features.info()

Let's look at a small subset of rows and columns to get a better picture.

In [ ]:
features.iloc[10:15, 5:7]

Labels were pre-generated by thresholding the K562 column of CAGE expression values, resulting in a relatively small percent of CAGE peaks with non-zero expression.  This will later demonstrate the impact of class imbalance.

Labels use genomic coordinates for their index, and re-indexed so their row order matches that of the features.

In [ ]:
labels = (
    pd.read_csv('../data/cage-k562-labels-hg38.csv', index_col = 0, squeeze = True)
    .reindex(features.index)
)

In [ ]:
labels.sample(10, random_state = 4)

In [ ]:
labels.value_counts()

Next, we extract just the chromosome of each region to later demonstrate the effect of cross-chromosome validation versus k-fold cross-validation with random shuffling.  This uses a regular expression to capture the text before a colon, for example the `chr1` in `chr1:110404708-110405059`.

In [ ]:
chroms = (
    features
    .index
    .to_series()
    .str.extract('^([^:]+)')
)

We first define a stratified k-fold cross-validation object.  Examples will be randomly shuffled without respect to the chromosome they are located on, and then split into two training/test sets while attempting to presereve the proportion of classes.

In [ ]:
shuffled_cv = StratifiedKFold(
    n_splits = 2,
    shuffle = True,
    random_state = 0
)

We next define cross-chromosome validation, which splits the data into two training/test sets where all regions on the same chromosome will be in one set or the other.  This prevents non-independent regions on the same chromosome from inflating performance by being present in both the training and test sets, which violates the Independent and Identically Distributed (IID) assumption of most statistical models.

In [ ]:
cc_cv = GroupKFold(n_splits = 2)

We then define multiple performance metrics ("scorers") for later use, using scikit-learn's names corresponding to area under the Precision-Recall curve (auPR), area under the Receiver Operating Characteristic curve (auROC), and accuracy.

In [ ]:
scorers = ['average_precision', 'roc_auc', 'accuracy']

We next create several pipelines which define a series of steps used to process data and fit a model.  scikit-learn commonly uses "estimator" as the variable name for pipelines, so we follow this convention.  I'll use the variable name model for the actual classifier that is the last step of an estimator.

Our baseline estimator uses a `DummyClassifier` to always predict the most common class (in our case, 0), and its performance can later be compared to our real model to see if it's learning anything from the data.

In [ ]:
baseline_estimator = make_pipeline(
    DummyClassifier(strategy = 'most_frequent')
)

scores = cross_validate(
    baseline_estimator,
    features,
    labels,
    groups = chroms,
    cv = cc_cv,
    scoring = scorers
)

scores['test_average_precision'].mean()

The baseline model is not learning anything, as we'd expect.  What if we looked at accuracy instead of auPR?

In [ ]:
scores['test_accuracy'].mean()

Accuracy is high compared to auPR because this is a highly imbalaned dataset, primarily composed of negatives.  This is why auPR or other metrics are preferred for imbalanced datasets&mdash;they aren't high simply due to class balance&mdash;though the "best" metric depends on the problem and your goals.

Next, we make several estimators using real models to demonstrate their correct evaluation under various data transformations.  We begin by defining two versions of a linear model: one using an L2 ("ridge") penalty which shrinks coefficients towards (but not equal to) zero, and another using an L1 ("LASSO") penalty which will select the most predictive features by setting some coefficients equal to zero.

In [ ]:
linear_model = LogisticRegression(penalty = 'l2', solver = 'liblinear', random_state = 0)
selection_linear_model = LogisticRegression(penalty = 'l1', solver = 'liblinear', random_state = 0)

We can now use these models to create estimators that perform data transformations before model fitting.

In [ ]:
linear_estimator = make_pipeline(
    VarianceThreshold(),
    StandardScaler(),
    linear_model
)

correct_balancing_linear_estimator = make_pipeline(
    RandomUnderSampler(random_state = 0),
    VarianceThreshold(),
    StandardScaler(),
    linear_model
)

`VarianceThreshold` is our first transformation and removes zero-variance features that generate warnings or cause problems for some models.

`StandardScaler` subtracts the mean of each feature and divides by its standard deviation, converting it into units of "standard deviations from the mean" which are then comparable across features.  This transformation is commonly applied with linear models, and placing it inside an estimator allows scaling the features without leaking information between the training and test sets.

For some of our evaluations, we use the `RandomUnderSampler` class from the imbalanced-learn package to create an equal number of positive and negative training examples by downsampling the negatives.  This generally improves predictive performance on the positive minority class by preventing the model from overfitting to the majority negative class.  As we explore later, a common pitfall balances the dataset outside of cross-validation, so that positives and negatives are balanced in both the training and test data.  `RandomUnderSampler`ensures only the training set is balanced, and not the test set, so that performance is estimated on the natural distribution of classes.

Next, we make two nearly identical estimators but use the linear model with L1 feature selection as the last step.

In [ ]:
correct_selection_linear_estimator = make_pipeline(
    VarianceThreshold(),
    StandardScaler(),
    selection_linear_model
)

correct_balancing_selection_linear_estimator = make_pipeline(
    RandomUnderSampler(random_state = 0),
    VarianceThreshold(),
    StandardScaler(),
    selection_linear_model
)

Finally, we create "pitfall" versions of the previous estimators that transform data outside of cross-validation.  First, we use the L1 linear model to pre-select features before estimating performance.  `SelectFromModel` fits the L1 linear model and returns only those features that were determined most predictive.

In [ ]:
pitfall_selector = SelectFromModel(selection_linear_model, max_features = 50)
pitfall_selected_features = pitfall_selector.fit_transform(features, labels)

Independently, we downsample negative examples before performance estimation.  We perform this operation twice, once to downsample the features and labels, and the next to downsample the list containing the chromosomes of each training example.

In [ ]:
pitfall_balanced_features, pitfall_balanced_labels = (
    RandomUnderSampler(random_state = 0)
    .fit_resample(features, labels)
)

pitfall_balanced_chroms, pitfall_balanced_labels = (
    RandomUnderSampler(random_state = 0)
    .fit_resample(chroms, labels)
)

We then combine pitfalls by feeding the balanced features and labels to the feature selector.

In [ ]:
pitfall_balanced_selected_features = (
    pitfall_selector
    .fit_transform(pitfall_balanced_features, pitfall_balanced_labels)
)

Finally, we encode the parameters for evaluating several pitfall and non-pitfall scenarios by specifying the estimator we want to use, the features, the labels, the list of chromosomes corresponding to each training example, a description of the data processing steps, and a boolean indicating whether these steps were applied prior to cross-validation or inside the CV loop.

In [ ]:
evaluation_parameters = [
    (linear_estimator, features, labels, chroms, 'none', False),
    (correct_selection_linear_estimator, features, labels, chroms, 'feature_selection', False),
    (linear_estimator, pitfall_selected_features, labels, chroms, 'feature_selection', True),
    (correct_balancing_linear_estimator, features, labels, chroms, 'balance_classes', False),
    (correct_balancing_selection_linear_estimator, features, labels, chroms, 'balance_classes,feature_selection', False),
    (linear_estimator, pitfall_balanced_features, pitfall_balanced_labels, pitfall_balanced_chroms, 'balance_classes', True),
    (linear_estimator, pitfall_balanced_selected_features, pitfall_balanced_labels, pitfall_balanced_chroms, 'balance_classes,feature_selection', True)
]

These scenarios are evaluated by looping over `evaluation_parameters`,  passing the relevant variables to `cross_validate`, and storing the average performance estimate for 3 different metrics.

In [ ]:
all_scores = []

for e, f, l, g, processing_steps, outside_cv in evaluation_parameters:
    for cv_type, cv in [('shuffled', shuffled_cv), ('cross_chromosome', cc_cv)]:
        scores = cross_validate(
            e,
            f,
            l,
            groups = g,
            cv = cv,
            scoring = scorers,
            n_jobs = -1
        )

        all_scores.append([
            'linear',
            cv_type,
            processing_steps,
            outside_cv,
            scores['test_average_precision'].mean(),
            scores['test_roc_auc'].mean(),
            scores['test_accuracy'].mean()
        ])

pd.DataFrame(
    all_scores,
    columns = [
        'model_type',
        'cv_type',
        'processing_steps',
        'outside_cv',
        'aupr',
        'auroc',
        'accuracy'
    ]
)

This summary showcases many pitfalls of machine learning, both in general and with respect to applications in genomics.

First, note that accuracy is typically very high when area under the precision-recall (auPR) or the receiver operating characteristic (auROC) curves are low.  This is characteristic of problems with major class imbalance, as accuracy simply measures the percent of predictions that are correct without respect to their proportion in the dataset.  Thus, a "model" that always predicts 0 when 96% of the labels are 0 will have an accuracy of 96%.  However, it will never correctly predict the minority class, and this is often the class of interest in genomics.

Note that auROC can be high when auPR is low.  The ROC curve is formed by values of the true positive rate (tpr) and false positive rate (fpr) at different classifier thresholds:

$$tpr = \frac{tp}{tp + fn}$$
$$fpr = \frac{fp}{fp + tn}$$

Where tp = # true positives, fp = # false positives, tn = # true negatives, and fn = # false negatives.  Precision and recall are defined as:

$$\mathrm{precision} = \frac{tp}{tp + fp}$$
$$\mathrm{recall} = tpr = \frac{tp}{tp + fn}$$

For highly imbalanced problems in genomics, the majority negative class is often much easier to predict than the minority positive class.  In such cases, the number of true negatives will be high and drive the false positive rate very low, impacting auROC.  True negatives are not measured by precision or recall, so auPR is not inflated by easy-to-predict majority classes.  This makes auPR a better choice than auROC for some (but not all) problems in genomics.

Second, feature selection performed outside CV leads to higher performance compared to selection performed inside CV.  All labels are used when selecting features outside CV, allowing the model to indirectly "peak" at the labels for every test set when CV is performed. When performed *inside* CV, features are selected using the training data only and the model is then evaluated using those features on the test data.  The amount of improvement can be quite large; see [Feature Selection Outside Cross-Validation](Notebook%20C%20-%20Feature%20Selection%20Outside%20Cross-Validation.ipynb) for an even more dramatic example.

Another procedure performed outside CV balances the number of positive and negative classes. We use the approach of downsampling the majority class, though other approaches exist.  This inflates performance by creating a dataset with fewer opportunities for false positives that no longer reflects the performance we would expect when applying the model in practice.  Balancing should be done inside CV on the training set only; downsampling is useful to reduce computation time and help prevent the model from over-fitting the majority class, but the test set class distribution must remain unbalanced to provide an unbiased performance estimate.

Third, note that shuffled k-fold CV reports higher auPR than cross-chromosome validation.  Regions on the same chromosome may not be independent and could have highly correlated features (violating independence in the IID assumption), making it easier for a model that trains on one example to accurately predict the label of another correlated example.  Shuffling examples and creating training/test splits that ignore these potential correlations can inflate performance.  Using cross-chromosome validation, where all examples on the same chromosome are placed in either the training or test set, prevents this type of inflation.

auPR can be lower in cross-chromosome validation simply because class balance is different compared to random splits.  However, performance inflation can be caused by training examples that violate the identically-distributed portion of the IID assumption, having features that are distributed differently across chromosomes due to the underlying biology.  Our examples are not highly correlated; we will show the performance difference is likely due to differences in feature distributions across chromosomes.  We first test this hypothesis by building an adversarial classifier that predicts which chromosome an example originates from, rather than predicting the enhancer label.

In [ ]:
chromosome_labels = (
    features
    .index
    .to_series()
    .str.startswith('chr1:')
    .astype(int)
)

baseline_adversarial_scores = cross_val_score(
    baseline_estimator,
    features,
    chromosome_labels,
    cv = 5,
    scoring = 'average_precision'
)
print(f'baseline adversarial performance: {baseline_adversarial_scores.mean():.2f}')

adversarial_scores = cross_val_score(
    linear_estimator,
    features,
    chromosome_labels,
    cv = 5,
    scoring = 'average_precision'
)
print(f'adversarial performance: {adversarial_scores.mean():.2f}')

This is a modest improvement over random guessing, suggesting that the performance drop observed using cross-chromosome evaluation is at least partly due to differences in the distribution of features.

Since our features are binary, feature density plots won't be very interesting.  Instead, we perform a difference-of-proportions test for the values of the most predictive features, split by chromosome.

In [ ]:
linear_estimator.fit(features, labels)

coefs = pd.Series(
    linear_estimator.steps[-1][1].coef_[0],
    index = features.columns[linear_estimator.steps[0][1].get_support()]
)
coefs = (
    coefs
    .abs()
    .sort_values(ascending = False)
    .index
    .tolist()
)

features['chrom'] = chromosome_labels

for i in range(5):
    counts = (
        features
        .groupby('chrom')
        [coefs[i]]
        .agg([sum, len])
    )

    zstat, pvalue = proportions_ztest(counts['sum'], counts['len'])
    print(f'{zstat:.1f} {pvalue:.1e} {coefs[i]}')

The p-value for the difference-of-proportions test is significant for the top 5 features.  Repeating this test for the least predictive features shows no significant p-values.  In summary, features the model finds most predictive are also distributed somewhat differently across chromosomes, causing a moderate drop in performance when the model is forced to train on examples from only one chromosome and predict on another.